Assignment 4

Yiying Peng

1. What assumptions are made about the attributes in Naïve Bayes method of classification and why? What is Laplacian correction and why it is necessary?

Naïve Bayes assumes that all of the features in the dataset are equally important and independent to reduce the number of parameters.

Laplacian correction is a technique used to smooth categorical data.

2.What is a decision tree? Describe a strategy that can be used to avoid overfitting in a decision tree.

A decision tree is a decision support tool that uses a tree-like graph or model of decisions and their possible consequences, including chance event outcomes, resource costs, and utility. It is one way to display an algorithm that only contains conditional control statements.

Avoid overfitting: post-pruning using Chi-squared test that allows the tree to perfectly classify the training set, and then post prune the tree. 

3.How can a decision tree be converted into a rule set? Illustrate with an example. What are the advantages of the rule set representation over the decision tree representation?

Every leaf that classifies becomes a rule. Find the path from the root of the leaf, and all the conditions that the leaf satisfies together.

Take iris data as an example. If the decision tree is as following, we can get the rule set:

    if y < 0.6: species will be setosa./    
    if y >= 0.6 and y <= 1.7: species will be versicolor./
    if y >= 0.6 and y > 1.7 and x <= 4.5: species will be versicolor.
    
    and so on.
    
The advantages is that it's really clear and we can find the result in a short time.

In [ ]:
iris data decision tree:
                                y < 0.6
                                /     \
                          (yes)        (no)
                         setosa        y <= 1.7
                                       /      \
                                  (yes)        (no)
                                 x <= 4.5      versicolor
                                /      \
                           (yes)        (no)
                        versicolor      y <= 1.5
                                        /      \
                                   (yes)        (no)
                                virginica       versicolor

4.For cancer data set.. Use three different methods to train a classifier that predicts whether cancer is Malignant or Bening,: a naïve bayes, decision tree and Random Forest. Look at the six confusion matrices: for each method, one for the training set and one for the test set, which methods, if any, are underfitting or overfitting the training set. Which algorithm do you think performs better?

In [ ]:
# Install packages
install.packages("mlbench")
install.packages("e1071")
install.packages("caTools")
install.packages("caret")
install.packages("mice")
install.packages("rpart")
install.packages("randomForest")
library(mlbench) 
library(e1071)
library(caTools)
library(caret)
library(mice)
library(rpart)
library(randomForest)

In [15]:
# Loading data
data(BreastCancer)
summary(BreastCancer) 
# (We can find 16 NA (missing values) in our dataset.)

      Id             Cl.thickness   Cell.size     Cell.shape  Marg.adhesion
 Length:699         1      :145   1      :384   1      :353   1      :407  
 Class :character   5      :130   10     : 67   2      : 59   2      : 58  
 Mode  :character   3      :108   3      : 52   10     : 58   3      : 58  
                    4      : 80   2      : 45   3      : 56   10     : 55  
                    10     : 69   4      : 40   4      : 44   4      : 33  
                    2      : 50   5      : 30   5      : 34   8      : 25  
                    (Other):117   (Other): 81   (Other): 95   (Other): 63  
  Epith.c.size  Bare.nuclei   Bl.cromatin  Normal.nucleoli    Mitoses   
 2      :386   1      :402   2      :166   1      :443     1      :579  
 3      : 72   10     :132   3      :165   10     : 61     2      : 35  
 4      : 48   2      : 30   1      :152   3      : 44     3      : 33  
 1      : 47   5      : 30   7      : 73   2      : 36     10     : 14  
 6      : 41   3      : 28 

In [ ]:
# Data Cleaning
# Removing NA and ID from dataset
dataset_impute <- mice(BreastCancer[,2:10],  print = FALSE) 
# Adding Target class to the imputed dataset without NA
BreastCancer <- cbind(BreastCancer[,11, drop = FALSE], mice::complete(dataset_impute, 1)) 
# Checking summary of the modified dataset
summary(BreastCancer)

In [ ]:
> summary(BreastCancer)
       Class      Cl.thickness   Cell.size     Cell.shape  Marg.adhesion  Epith.c.size  Bare.nuclei   Bl.cromatin  Normal.nucleoli
 benign   :458   1      :145   1      :384   1      :353   1      :407   2      :386   1      :413   2      :166   1      :443    
 malignant:241   5      :130   10     : 67   2      : 59   2      : 58   3      : 72   10     :133   3      :165   10     : 61    
                 3      :108   3      : 52   10     : 58   3      : 58   4      : 48   2      : 31   1      :152   3      : 44    
                 4      : 80   2      : 45   3      : 56   10     : 55   1      : 47   5      : 30   7      : 73   2      : 36    
                 10     : 69   4      : 40   4      : 44   4      : 33   6      : 41   3      : 29   4      : 40   8      : 24    
                 2      : 50   5      : 30   5      : 34   8      : 25   5      : 39   8      : 21   5      : 34   6      : 22    
                 (Other):117   (Other): 81   (Other): 95   (Other): 63   (Other): 66   (Other): 42   (Other): 69   (Other): 69    
    Mitoses   
 1      :579  
 2      : 35  
 3      : 33  
 10     : 14  
 4      : 12  
 7      :  9  
 (Other): 17 

In [ ]:
# Splitting Dataset into training and testing
set.seed(150)    
split=sample.split(BreastCancer, SplitRatio = 0.7)  
training_set=subset(BreastCancer,split==TRUE)       
test_set=subset(BreastCancer,split==FALSE)              
# Removing target class 
topredict_set<-test_set[2:10]                       

In [ ]:
# Naive Bayes Classifier
model_naive<- naiveBayes(Class ~ ., data = training_set)   
preds_naive <- predict(model_naive, newdata = topredict_set)        
(conf_matrix_naive <- table(preds_naive, test_set$Class))       
confusionMatrix(conf_matrix_naive) 
# (The confusion matrix shows that Naive Bayes classifier predicted 129 benign cases correctly and 2 wrong predictions. 
# Similarly, the model predicted 72 malignant cases correctly and 6 wrong predictions.)
# (The accuracy of Naive Bayes Classifier is 96.17%)

In [ ]:
> (conf_matrix_naive <- table(preds_naive, test_set$Class))
           
preds_naive benign malignant
  benign       129         2
  malignant      6        72
> confusionMatrix(conf_matrix_naive)
Confusion Matrix and Statistics

           
preds_naive benign malignant
  benign       129         2
  malignant      6        72
                                         
               Accuracy : 0.9617         
                 95% CI : (0.926, 0.9833)
    No Information Rate : 0.6459         
    P-Value [Acc > NIR] : <2e-16         
                                         
                  Kappa : 0.9173         
                                         
 Mcnemar's Test P-Value : 0.2888         
                                         
            Sensitivity : 0.9556         
            Specificity : 0.9730         
         Pos Pred Value : 0.9847         
         Neg Pred Value : 0.9231         
             Prevalence : 0.6459         
         Detection Rate : 0.6172         
   Detection Prevalence : 0.6268         
      Balanced Accuracy : 0.9643         
                                         
       'Positive' Class : benign

In [ ]:
# Random Forest Classifier
model_rf <- randomForest(Class ~ ., data = training_set, importance=TRUE, ntree = 5)
preds_rf <- predict(model_rf, topredict_set)              
(conf_matrix_forest <- table(preds_rf, test_set$Class))
confusionMatrix(conf_matrix_forest)  
# (The confusion matrix shows that Random Forest classifier predicted 127 benign cases correctly and 3 wrong predictions. 
# Similarly, the model predicted 71 malignant cases correctly and 8 wrong predictions.)
# (The accuracy of Naive Bayes Classifier is 95.69%)

In [ ]:
> (conf_matrix_forest <- table(preds_rf, test_set$Class))
           
preds_rf    benign malignant
  benign       127         3
  malignant      8        71
> confusionMatrix(conf_matrix_forest)
Confusion Matrix and Statistics

           
preds_rf    benign malignant
  benign       127         3
  malignant      8        71
                                          
               Accuracy : 0.9474          
                 95% CI : (0.9078, 0.9734)
    No Information Rate : 0.6459          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.8867          
                                          
 Mcnemar's Test P-Value : 0.2278          
                                          
            Sensitivity : 0.9407          
            Specificity : 0.9595          
         Pos Pred Value : 0.9769          
         Neg Pred Value : 0.8987          
             Prevalence : 0.6459          
         Detection Rate : 0.6077          
   Detection Prevalence : 0.6220          
      Balanced Accuracy : 0.9501          
                                          
       'Positive' Class : benign

In [ ]:
# Decision Tree Classifier
model_dtree<- rpart(Class ~ ., data=training_set) 
preds_dtree <- predict(model_dtree,newdata=topredict_set, type = "class")
(conf_matrix_dtree <- table(preds_dtree, test_set$Class))
confusionMatrix(conf_matrix_dtree)
# (The confusion matrix shows that Decision Tree classifier predicted 127 benign cases correctly and 5 wrong predictions. 
# Similarly, the model predicted 69 malignant cases correctly and 8 wrong predictions.)
# (The accuracy of Naive Bayes Classifier is 93.78%)

In [ ]:
> (conf_matrix_dtree <- table(preds_dtree, test_set$Class))
           
preds_dtree benign malignant
  benign       127         5
  malignant      8        69
> confusionMatrix(conf_matrix_dtree)
Confusion Matrix and Statistics

           
preds_dtree benign malignant
  benign       127         5
  malignant      8        69
                                         
               Accuracy : 0.9378         
                 95% CI : (0.896, 0.9665)
    No Information Rate : 0.6459         
    P-Value [Acc > NIR] : <2e-16         
                                         
                  Kappa : 0.8652         
                                         
 Mcnemar's Test P-Value : 0.5791         
                                         
            Sensitivity : 0.9407         
            Specificity : 0.9324         
         Pos Pred Value : 0.9621         
         Neg Pred Value : 0.8961         
             Prevalence : 0.6459         
         Detection Rate : 0.6077         
   Detection Prevalence : 0.6316         
      Balanced Accuracy : 0.9366         
                                         
       'Positive' Class : benign         

In [ ]:
Comparing the accuracy of these three models,
we found that Naive Bayes Classifier Algorithm is a better Classifier.